# 第八章 添加提示词和资源功能

在前面的课程中，您创建了一个仅提供工具的MCP服务器。在本课程中，您将获得研究服务器文件的更新文件，该文件现在除了提供2个工具外，还提供了一个提示模板和2个资源功能。您还将获得mcp聊天机器人文件的更新文件，mcp客户端在其中公开提示词（prompt）和资源（resources）供用户使用。这些文件位于 mcp_project 文件夹中。

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> 要访问  <code>mcp_project</code> 文件夹 :</b> 1) 单击笔记本顶部菜单上的 <em>"File"</em> 选项，然后 2) 点击 <em>"Open"</em> ，最后 3) 点击 <em>L7</em>.
</div>

## 在MCP服务器中定义资源和提示 - 可选阅读


您可选择在观看视频前或后阅读本节内容。您可以跳转到笔记本的末尾来运行更新的聊天机器人。

**资源**

您从第三章中了解到，研究服务器将研究论文的信息保存在一个名为 papers_info.json 的 json 文件中，该文件存储在一个标有主题名称的文件夹下。所有主题都存储在 papers 目录下。如果您查看 mcp_project 下提供给您的papers 文件夹，你会发现两个标有 ai_interpretability 和 llm_reasoning 的文件夹，在每个文件夹中，您都有 papers_info.json 文件。

资源是 MCP 服务器可以向 LLM 应用程序公开的只读数据。资源类似于REST API中的GET端点——它们提供数据，但不应执行显著的计算或产生副作用。例如，资源可以是目录中的文件夹列表，也可以是文件夹中的文件内容。这里，MCP服务器提供两种资源：
- 论文目录下的可用主题文件夹列表；
- 存储在主题文件夹下的论文信息。

以下一个代码片段，显示了如何再次使用FastMCP（使用装饰器 @MCP.resource（uri））在MCP服务器中定义资源。您可以在 mcp_project 文件夹中找到完整的代码。@mcp.resources() 中定义的URI用于唯一标识资源，作为服务器开发人员，您可以自定义URI。但一般来说，它遵循以下原则：sth://xyz/xcv .在这个例子中，使用了两种类型的URI：
- 静态URI:papers://folders（表示可用主题列表）
- 动态URI:papers://{topic}（表示客户端在运行时指定的主题下的论文信息）


``` python
@mcp.resource("papers://folders")
def get_available_folders() -> str:
    """
    列出论文目录中所有可用的主题文件夹。

    此资源提供了所有可用的主题文件夹的一个简表。
    """
    folders = []
    
    # Get all topic directories
    if os.path.exists(PAPER_DIR):
        for topic_dir in os.listdir(PAPER_DIR):
            topic_path = os.path.join(PAPER_DIR, topic_dir)
            if os.path.isdir(topic_path):
                papers_file = os.path.join(topic_path, "papers_info.json")
                if os.path.exists(papers_file):
                    folders.append(topic_dir)
    
    # 创建一个简单的 markdown 列表
    content = "# Available Topics\n\n"
    if folders:
        for folder in folders:
            content += f"- {folder}\n"
        content += f"\nUse @{folder} to access papers in that topic.\n"
    else:
        content += "No topics found.\n"
    
    return content

@mcp.resource("papers://{topic}")
def get_topic_papers(topic: str) -> str:
    """
    获取特定主题的论文详细信息。

    参数（Args）：
    topic：要检索论文的研究主题
    
    """
    topic_dir = topic.lower().replace(" ", "_")
    papers_file = os.path.join(PAPER_DIR, topic_dir, "papers_info.json")
    
    if not os.path.exists(papers_file):
        return f"# No papers found for topic: {topic}\n\nTry searching for papers on this topic first."
    
    try:
        with open(papers_file, 'r') as f:
            papers_data = json.load(f)
        
        # 创建包含论文详细信息的 Markdown 内容
        content = f"# Papers on {topic.replace('_', ' ').title()}\n\n"
        content += f"Total papers: {len(papers_data)}\n\n"
        
        for paper_id, paper_info in papers_data.items():
            content += f"## {paper_info['title']}\n"
            content += f"- **Paper ID**: {paper_id}\n"
            content += f"- **Authors**: {', '.join(paper_info['authors'])}\n"
            content += f"- **Published**: {paper_info['published']}\n"
            content += f"- **PDF URL**: [{paper_info['pdf_url']}]({paper_info['pdf_url']})\n\n"
            content += f"### Summary\n{paper_info['summary'][:500]}...\n\n"
            content += "---\n\n"
        
        return content
    except json.JSONDecodeError:
        return f"# Error reading papers data for {topic}\n\nThe papers data file is corrupted."
```

提示词模板

服务器还可以提供提示词模板。您可以使用装饰器 @MCP.prompt() 在 MCP 服务器中定义此功能，如下面的代码片段所示。MCP 将使用 generate_search_prompt 作为提示词名称，从函数的参数推断提示参数，从文档字符串推断提示描述。


Server can also provide a prompt template. You can define this feature in the MCP server using the decorator `@mcp.prompt()` as shown in the code snippet below. MCP will use `generate_search_prompt` as the prompt name, infer the prompt arguments from the function's argument and the prompt's description from the doc string.

```python
@mcp.prompt()
def generate_search_prompt(topic: str, num_papers: int = 5) -> str:
    """为 Claude 生成提示，以查找和讨论特定主题的学术论文。"""
    return f"""使用search_papers工具搜索关于'{topic}'的{num_papers}篇学术论文。

    请遵循以下指令:
       
    3. 提供一个全面的总结，包括：
       - '{topic}'当前研究状态的概述
       - 论文之间的共同主题和趋势
       - 研究差距或未来研究方向
       - 该领域最具影响力或影响力的论文
    
    4. 以清晰、结构化的格式呈现你的发现，使用标题和列表项以便于阅读。

    请同时提供每篇论文的详细信息以及{topic}研究领域的整体概述。
    """
    

## 使用 MCP 聊天机器人中的资源和提示-可选阅读

聊天机器人更新使用户能够使用斜线命令与提示进行交互：
- 用户可以使用命令 /提示词 列出可用的提示
- 用户可以使用命令/提示词 <name><arg1=value1> 调用特定的提示符

聊天机器人还允许用户使用 @ 字符与资源进行交互：
- 用户可以使用命令 @文件夹 查看可用主题
- 用户可以使用命令 @主题 获取该主题下的论文信息

确保检查 mcp_project 文件夹中的更新代码。有几个新添加的方法（get_resource、execute_prompt、list_prompts）。以下是添加到聊天机器人的更新的简要总结：

Make sure to check the updated code in the `mcp_project` folder. There's a couple of newly added methods (`get_resource`, `execute_prompt`, `list_prompts`). Here's a brief summary of the updates added to the chatbot:


- 在 connect_to_server 中：客户端向服务器请求列出它们提供的资源和提示模板（除了工具请求）。资源 URI 和提示词名称都被存储在 MCP 聊天机器人中。

    <img src="images/ch08_resource_discovery.png" width="400">


    <img src="images/ch08_prompt_discovery.png" width="400">


- 在 chat_loop 中：会检查用户的输入，查看用户是否使用了任何斜线命令或 @ 选项。

- 如果用户键入：@文件夹 或 @主题 ，则会调用新添加的方法 get_resource ，将请求发送到服务器。
   
    <img src="images/ch08_resource_invocation.png" width="400">

- 如果用户输入：`/提示词`，则会调用新添加的方法 `list_prompts`。

- 如果用户输入：`/提示词 <name> <arg1=value1>`，则会调用新添加的方法 `execute_prompt`，并将请求发送到服务器：
   
   <img src="images/ch08_prompt_invocation.png" width="400">
   
   将提示词传递到LLM。
- 否则，查询将由 LLM 处理。

## 运行MCP聊天机器人

**终端指令**

- 要打开终端，请运行以下命令。
- 打开以下终端。
- 导航到 `mcp_project` 目录：
- `cd L7/mcp_project`
- 激活虚拟环境：
- `source .venv/bin/activate`
- 运行聊天机器人：
- `uv run mcp_chatbot.py`
- 要退出聊天机器人，请输入 `quit`。
- 如果您运行一些查询并希望访问 `papers` 文件夹或任何输出文件：1) 点击笔记本顶部菜单上的 `File` 选项，2) 点击 `Open`，然后 3) 点击 `L7` -> `mcp_project`。


如果想要在 jupyter 中使用 terminal 的话请运行下边这段代码


确保与聊天机器人进行交互。以下是一些查询示例：
- **@文件夹*
- **@ai_可解释性**
- **/提示词**
- **/提示词 产生搜索提示词 主题=历史 论文数量=2**


不同的运行结果：由于 AI 聊天模型的动态概率特性，其每次执行生成的输出结果可能会有所不同。如果您的结果与视频中显示的结果不同，请不要感到惊讶。

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>不同的运行结果:</b> 由于 AI 聊天模型的动态概率特性，其每次执行生成的输出结果可能会有所不同。如果您的结果与教程中显示的结果不同，请不要感到惊讶。</p>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>下载 Notebooks:</b> 1) 点击笔记本顶部菜单上的 <em>"File"</em> ，然后  2) 点击 <em>"下载为"</em> 并选择 <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; 如需更多帮助，请参阅 <em>"附录 - 提示、帮助和下载"</em> 课程.</p>

</div>